In [105]:
# Start by importing the necessary libraries

import requests # to make the API request for obtaining data
import pandas as pd # to work with dataframes
import json # to parse the JSON data

In [106]:
# Function that gets a genre and returns the data for that genre from the API

def genreData(genre, url="http://localhost:8000/genre"):
    '''
    Function that takes a genre and returns the data for that genre from the API

    Parameters:
    genre (str): The genre for which the data is to be obtained
    url (str): The URL of the API to be used

    Returns:
    results (json): Response from the API for the genre
    '''

    # Make the API request
    response = requests.get(url, params={"genre": genre})

    # Check if the request was successful
    if response.status_code == 200:
        results = response.json().get("results", [])
        return results
    else:
        print(f"Failed to fetch data for genre: {genre}")
        return []

In [107]:
# Fetch data for multiple genres

genres = ["pop", "rock", "jazz", "hip hop", "classical"]
all_tracks = []

for genre in genres:
    genre_data = genreData(genre)
    for track in genre_data:
        track["genre"] = genre
    all_tracks.append(genre_data)

In [108]:
# Convert the data to a pandas dataframe

df = pd.DataFrame([track for genre_data in all_tracks for track in genre_data])

# Display the first few rows of the dataframe

print(df.head())


                       id                                   name  \
0  0WbMK4wrZ1wFSty9F7FCgu                       Good Luck, Babe!   
1  6dOtVTDdiauQNBQEDOtlAB                     BIRDS OF A FEATHER   
2  5N3hjp1WNayUPZrA8kJmJP                   Please Please Please   
3  2qSkIjg1o9h3YT9RAgYN75                               Espresso   
4  7221xIgOnuakPdLqT0F3nP  I Had Some Help (Feat. Morgan Wallen)   

                        artists genre  
0               [Chappell Roan]   pop  
1               [Billie Eilish]   pop  
2           [Sabrina Carpenter]   pop  
3           [Sabrina Carpenter]   pop  
4  [Post Malone, Morgan Wallen]   pop  


In [109]:
# Function that gets each id from the df and returns the features for that id from the API

def trackFeatures(track_id, url="http://localhost:8000/features"):
    '''
    Function that takes a track id and returns the features for that track from the API
    Parameters:
    track_id (str): The track id for which the features are to be obtained
    url (str): The URL of the API to be used
    Returns:
    dict: A dictionary of features for the track
    '''
    response = requests.get(url, params={"track_id": track_id})

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for track: {track_id}")
        return {}

In [110]:
# Fetch features for all tracks, create a dataframe, mix it with the original dataframe and print the head

all_features = []

for track_id in df['id']:
    track_features = trackFeatures(track_id)
    all_features.append(track_features)

df_features = pd.DataFrame(all_features)
print(df_features.head())

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.700   0.582   11    -5.960     0       0.0356       0.05020   
1         0.747   0.507    2   -10.171     1       0.0358       0.20000   
2         0.669   0.586    9    -6.073     1       0.0540       0.27400   
3         0.701   0.760    0    -5.478     1       0.0285       0.10700   
4         0.638   0.855    7    -4.860     1       0.0264       0.00757   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000000    0.0881    0.785  116.712  audio_features   
1          0.060800    0.1170    0.438  104.978  audio_features   
2          0.000000    0.1040    0.579  107.071  audio_features   
3          0.000065    0.1850    0.690  103.969  audio_features   
4          0.000000    0.2450    0.731  127.986  audio_features   

                       id                                   uri  \
0  0WbMK4wrZ1wFSty9F7FCgu  spotify:track:0WbMK4wrZ1wFSty9F7FCgu   
1  6dOtVTDdi

In [111]:
# Save the dataframe to a CSV file

df_final = pd.merge(df, df_features, on='id', how='left')
print(df_final.head())
df_final.to_csv("music_data.csv", index=False)

                       id                                   name  \
0  0WbMK4wrZ1wFSty9F7FCgu                       Good Luck, Babe!   
1  6dOtVTDdiauQNBQEDOtlAB                     BIRDS OF A FEATHER   
2  5N3hjp1WNayUPZrA8kJmJP                   Please Please Please   
3  2qSkIjg1o9h3YT9RAgYN75                               Espresso   
4  7221xIgOnuakPdLqT0F3nP  I Had Some Help (Feat. Morgan Wallen)   

                        artists genre  danceability  energy  key  loudness  \
0               [Chappell Roan]   pop         0.700   0.582   11    -5.960   
1               [Billie Eilish]   pop         0.747   0.507    2   -10.171   
2           [Sabrina Carpenter]   pop         0.669   0.586    9    -6.073   
3           [Sabrina Carpenter]   pop         0.701   0.760    0    -5.478   
4  [Post Malone, Morgan Wallen]   pop         0.638   0.855    7    -4.860   

   mode  speechiness  ...  instrumentalness  liveness  valence    tempo  \
0     0       0.0356  ...          0.000000    